# file sizes, number of hits and cdf

In [ ]:
import glob
filenames = glob.glob('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/scores/RC_2016-10.bz2.decompressed/*txt')

In [ ]:
sims = []

In [ ]:
for name in filenames:
    try:
        with open(name , 'r') as content_file:
            content = content_file.read()
            json_data = content.replace('][',',')
            j_object = json.loads(json_data)
            j_df = pd.DataFrame(j_object)
            counts = collections.Counter(j_df['tweet_id'].tolist())
            counts_sorted = counts.most_common()
            c_df = pd.DataFrame(counts_sorted, columns=['tweet_id', 'frequency'])
            final_df = final_df.append(c_df, ignore_index = True) 
            final_df['t_f'] = final_df.groupby(['tweet_id'])['frequency'].transform('sum')
            final_df = final_df.drop_duplicates(subset=['tweet_id'])
            del final_df['frequency']
            final_df.columns = ['tweet_id', 'frequency']
#             print(final_df.head())
#             print(len(final_df))
            i = i + 1
            print(i)
    except:
        no_hits_files = no_hits_files + 1

In [ ]:
print(no_hits_files)
print(hits_files)
print(no_of_hits)

In [ ]:
import json
import pandas as pd
with open('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/scores/RC_2016-10.bz2.decompressed/7076_scores_stsb.txt' , 'r') as content_file:
        size = content_file.tell()
        content = content_file.read()
        json_data = content.replace('][',',')
        j_object = json.loads(json_data)
        j_df = pd.DataFrame(j_object)
        print(j_df.shape[0])

# CDF plot and theme distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import collections
counts, bin_edges = np.histogram (sims, bins=20, normed=False)
cdf = np.cumsum (counts)
plt.plot(bin_edges[1:], cdf/cdf[-1])
plt.xlabel('cosine similarity')
plt.ylabel('3M hits')

In [ ]:
data = pd.read_csv('./clustering/annotated/combined_annotated_updated1.csv')

In [ ]:
relevant_data = data[data['relevance']==1]

In [ ]:
relevant_data.head()

In [ ]:
count = 0
counts = []
labels = ['Muslims', 'Racism', 'Elections','Michel Obama', 'Congress','Islam', 'Zimbabwe', 'Cop Violence',
         'White House', 'UK', 'Republicans', ' Barack Obama', 'Hilary Clinton', 'Trump', 'Protests/Rallies',
         'Biden', 'Democratics', 'US Gov', 'Gen Politics', 'LGBTQ', 'ISIS', 'China', 'Kenya', 'Russia', 'Korea',
         'Syria', 'Abortion', 'Miscellaneous','Sanders/Cruz','Israel','Liberals', 'Iran', 'Germany', 'Greek', 
          'Flint water crisis','Turkey','Pakistan','Bill Cosby','Gun law', 'James Comey','Climate change',
          'India','Columbia','Ferguson','No cyber censorship movement',
          'Brazil','Saudi -Yemen','Belgium','Panama','Religion','Europe','Mexico','Baltimore',
          'Paris climate deal','Afghan  - Taliban','Australia','Cuba','Italy']
for i in range(1,59):
    shape = relevant_data[relevant_data['theme'] == i].shape
    counts.append(shape[0])
    count =  count + shape[0]
print(count)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.style.use('ggplot')
df = pd.DataFrame()
df['counts'] = counts
df['labels'] = labels
df = df.sort_values('counts', ascending=False)
df.set_index('labels', inplace=True)
fig, ax = plt.subplots(figsize=(7,10))
df.plot(kind='barh', legend = False, ax=ax)
ax.set_xlabel('Number of Tweets')
ax.set_ylabel('Themes')

# Temporal Analysis

In [ ]:
import numpy as np 
import pandas as pd
import csv
import os
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import random
import emoji
import string
import json
import nltk

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

# list of all the dataset files
dataset_paths = ["../../datasets/russia_052020_tweets_csv_hashed_2.csv", 
         "../../datasets/russian_linked_tweets_csv_hashed.csv", 
         "../../datasets/ira_tweets_csv_hashed.csv", 
         "../../datasets/russia_201906_1_tweets_csv_hashed.csv"]

# path to store the entire combined dataset
combined_dataset_path = "../datasets/russian_trolls.csv"

# returns a pandas dataframe consisting of entries from all the dataset files
def get_combined_dataset(paths):
    data = pd.concat((pd.read_csv(file) for file in tqdm(paths)))
    return data

data = get_combined_dataset(dataset_paths)
print("Number of tweets in the dataset: ", data.shape[0])


In [ ]:
data.columns

# Get samples for annotation

In [ ]:
final_annotations = pd.DataFrame()

In [ ]:
adf = pd.read_csv('./annotations/2016.11.annotation_data_point_95.csv')

In [ ]:
adf.shape

In [ ]:
res = adf[~adf[['tweet', 'matching_sentence']].apply(frozenset, axis=1).duplicated()]


In [ ]:
res = res[~res[['tweet']].apply(frozenset, axis=1).duplicated()]

In [ ]:
res = res[~res[['matching_sentence']].apply(frozenset, axis=1).duplicated()]

In [ ]:
res.shape

In [ ]:
final_annotations = final_annotations.append(res.sample(n = 10))

In [ ]:
final_annotations.shape

In [ ]:
del final_annotations['Unnamed: 0']

In [ ]:
fdf = final_annotations.sample(n=170)

In [ ]:
fdf['annotation_based_on_sentence'] = 2
fdf['annotation_based_on_post'] = 2

In [ ]:
fdf.shape

In [ ]:
pdf = pd.read_csv('./annotation_samples_saved.csv')

In [ ]:
fdf = pdf.append(fdf, ignore_index=True)

In [ ]:
pdf.shape

In [ ]:
fdf.to_csv('./annotations/annotation_samples.csv')

# Threshold matrix

In [ ]:
odf = fdf

In [ ]:
odf.head()

In [ ]:
df = pd.read_csv('./clustering/annotated/annotation_samples_aishwarya.csv')

In [ ]:
odf['annotation_based_on_sentence'] = df['annotation_based_on_sentence'].values

In [ ]:
odf['annotation_based_on_post'] = df['annotation_based_on_post'].values

In [ ]:
print(odf[(odf['sim'] > 0.65) & (odf['sim'] <= 0.7)].shape)
print(odf[(odf['sim'] > 0.7) & (odf['sim'] <= 0.75)].shape)
print(odf[(odf['sim'] > 0.75) & (odf['sim'] <= 0.8)].shape)
print(odf[(odf['sim'] > 0.8) & (odf['sim'] <= 0.85)].shape)
print(odf[(odf['sim'] > 0.85) & (odf['sim'] <= 0.9)].shape)
print(odf[(odf['sim'] > 0.9) & (odf['sim'] <= 0.95)].shape)
print(odf[(odf['sim'] > 0.95) & (odf['sim'] <= 1.0)].shape)

In [ ]:
odf.to_csv('./annotation_samples_saved.csv')

In [ ]:
odf = pd.read_csv('./annotation_samples_aishwarya.csv')

In [ ]:
sdf = odf[odf['annotation_based_on_post'].notnull()]

In [ ]:
import numpy as np
matrix_0 = []
matrix_1 = []
from sklearn.metrics import classification_report
target_names = ['irrelevant', 'relevant']
for threshold in np.arange(0.65, 1.0, 0.05):
    sdf['true-values'] = np.where(sdf['sim'] > threshold, 1, 0)
    c = classification_report(sdf['true-values'],
                              sdf['annotation_based_on_post'], target_names=target_names, output_dict = True)
    matrix_0.append({'threshold': threshold, 'class': '0',
                   'precision' : (c['irrelevant'])['precision'],
                  'recall': (c['irrelevant'])['recall'],
                  'f1-score': (c['irrelevant'])['f1-score'],
                  'support': (c['irrelevant'])['support'],
                  'accuracy': c['accuracy']})
    matrix_1.append({'threshold': threshold, 'class': '1',
                   'precision' : (c['relevant'])['precision'],
                  'recall': (c['relevant'])['recall'],
                  'f1-score': (c['relevant'])['f1-score'],
                  'support': (c['relevant'])['support'],
                  'accuracy': c['accuracy']})
#     print(c)

In [ ]:
k = pd.DataFrame(matrix_1)
del k['support']
k

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import collections
counts, bin_edges = np.histogram (sims, bins=500, normed=False)
cdf = np.cumsum (counts)
plt.plot(bin_edges[1:], cdf/cdf[-1])
plt.xlabel('ratio of post length to max tweet length')
plt.ylabel('randomized hits')

In [ ]:
def plot_cdf(list_counts, xlabel, path, leg=False, islogx=True, xlimit=False):
    t_col = "#235dba"
    g_col = "#005916"
    c_col = "#a50808"
    r_col = "#ff9900"
    black = "#000000"
    pink = "#f442f1"
    t_ls = '-'
    r_ls = '--'
    c_ls = ':'
    g_ls = '-.'

    markers = [".", "o", "v", "^", "<", ">", "1", "2"]
    colors = [t_col, c_col, g_col, r_col, black, 'c', 'm', pink]
    line_styles = [t_ls, r_ls, c_ls, g_ls,t_ls, r_ls, c_ls, g_ls, t_ls]
    colors = colors[1:]
    line_styles= line_styles[1:]
    while(len(list_counts) > len(colors)):
        colors = colors + shuffle(colors)
        line_styles = line_styles + shuffle(line_styles)
        
    if xlimit:
        l2 = []
        for l in list_counts:
            l2_1 = [x for x in l if x<=xlimit]
            l2.append(l2_1)
        list_counts = l2
    
    for l in list_counts:
        l.sort()
    fig, ax = plt.subplots(figsize=(6,4))
    yvals = []
    for l in list_counts:
        yvals.append(np.arange(len(l))/float(len(l)-1))
    for i in range(len(list_counts)):
        ax.plot(list_counts[i], yvals[i], color=colors[i], linestyle=line_styles[i])
    if islogx:
        ax.set_xscale("log")
    plt.xlabel(xlabel)
    plt.ylabel('CDF')
    plt.grid()
    for item in ([ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(13)
    
    if leg:
        plt.legend(leg, loc='best', fontsize=13)
    
    plt.show()
    fig.savefig(path, bbox_inches='tight')


In [ ]:
plot_cdf([sims], 'ratio', './cdf_of_ratio_of_tweet_lengths.pdf', islogx=False)

# Getting samples with sim  > 0.8 along with the time difference

In [ ]:
combined_annotation = pd.DataFrame()

In [ ]:
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2016.10.annotation_data_point_8.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2016.10.annotation_data_point_85.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2016.10.annotation_data_point_9.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2016.10.annotation_data_point_95.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2016.11.annotation_data_point_8.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2016.11.annotation_data_point_85.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2016.11.annotation_data_point_9.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2016.11.annotation_data_point_95.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2017.04.annotation_data_point_8.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2017.04.annotation_data_point_85.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2017.04.annotation_data_point_9.csv'), ignore_index = True)
combined_annotation = combined_annotation.append(pd.read_csv('./annotations/2017.04.annotation_data_point_95.csv'), ignore_index = True)

res = combined_annotation[~combined_annotation[['tweet_text', 'matching_sentence']].apply(frozenset, axis=1).duplicated()]
res = res[~res[['tweet_text']].apply(frozenset, axis=1).duplicated()]
res = res[~res[['matching_sentence']].apply(frozenset, axis=1).duplicated()]

In [ ]:
res = combined_annotation

In [ ]:
fdata = data[data['tweetid'].isin(res['tweet_id'].to_list())]

In [ ]:
times = []
for idx, row in res.iterrows():
    element = fdata[fdata['tweetid'] == row['tweet_id']]
    time = element['tweet_time'].to_string()
    time = time.split()
#     print(time[1]+' '+time[2])
    times.append(time[1]+' '+time[2])

In [ ]:
# del res['tweet_time']
res['tweet_time'] = times

In [ ]:
rel_posts = pd.DataFrame()

In [ ]:
posts = pd.read_csv('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/posts/posts-2016-10.csv')
fposts = posts[posts['id'].isin(res['post_id'].to_list())]
rel_posts = rel_posts.append(fposts, ignore_index=True)
posts = pd.read_csv('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/posts/posts-2016-11.csv')
fposts = posts[posts['id'].isin(res['post_id'].to_list())]
rel_posts = rel_posts.append(fposts, ignore_index=True)
posts = pd.read_csv('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/posts/posts-2017-04.csv')
fposts = posts[posts['id'].isin(res['post_id'].to_list())]
rel_posts = rel_posts.append(fposts, ignore_index=True)
                             

In [ ]:
import datetime
times = []
for idx, row in res.iterrows():
    element = rel_posts[rel_posts['id'] == row['post_id']]
    print((element['subreddit'].values))
    time = element['created_utc'].astype('int64')
    time = str(time).split()
#     print(time)
    try:
        times.append(datetime.datetime.fromtimestamp(int(time[1])))
    except ValueError:
        print(time[0])
        times.append(time[0])

In [ ]:
# del res['post_time']
res['post_time'] = times

In [ ]:
res = res[res['post_time'] !=  'Series([],']

In [ ]:
t = pd.to_datetime(res.tweet_time, infer_datetime_format=True)

In [ ]:
del res['tweet_time']
res['tweet_time'] = t

In [ ]:
t = pd.to_datetime(res.post_time, infer_datetime_format=True)
del res['post_time']
res['post_time'] = t

In [ ]:
td = (res.post_time - res.tweet_time).astype('timedelta64[h]')

In [ ]:
res['time_difference'] = td

In [ ]:
res.to_csv('./annotations/combinedpoint8.csv')

# CDF of time difference between post and tweets of all the hits so far

In [ ]:
combined_times =pd.DataFrame()

In [ ]:
twitter_data = pd.read_csv('./relevant_tweets.csv')
twitter_data.shape

In [ ]:
reddit_data = pd.read_csv('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/posts/posts-2017-04.csv')
reddit_data.shape

In [ ]:
hits = pd.read_csv('./results/combined_hits_2017.04.csv')

In [ ]:
hits = hits.rename(columns={'tweet_time': 'post_time'})
del hits['Unnamed: 0']

In [ ]:
hits.dtypes

In [ ]:
hits = hits[hits['post_time'] != 'Series([],']

In [ ]:
hits.shape

In [ ]:
fdata = data[data['tweetid'].isin(hits['tweet_id'].unique())]

In [ ]:
fdata.shape

In [ ]:
times = []
authors = []
for idx, row in hits.iterrows():
    element = fdata[fdata['tweetid'] == row['tweet_id']]
    time = element['tweet_time'].to_string()
    time = time.split()
#     print(time[1]+' '+time[2])
    times.append(time[1]+' '+time[2])
#     authors.append(row[''])

In [ ]:
len(times)

In [ ]:
hits['tweet_time'] = times

In [ ]:
hits[hits['tweet_time'] == 'Series([],'].shape

In [ ]:
t = pd.to_datetime(hits.tweet_time, infer_datetime_format=True)
del hits['tweet_time']
hits['tweet_time'] = t


In [ ]:
t = pd.to_datetime(hits.post_time, infer_datetime_format=True)
del hits['post_time']
hits['post_time'] = t

In [ ]:
td = (hits.post_time - hits.tweet_time).astype('timedelta64[h]')

In [ ]:
hits['td'] = td

In [ ]:
combined_times = combined_times.append(hits)

In [ ]:
td = (combined_times.post_time - combined_times.tweet_time).astype('timedelta64[h]')
del combined_times['td']
combined_times['td'] = td

In [ ]:
combined_times[combined_times['td'] == 0].shape

In [ ]:
combined_times.head()

In [ ]:
plot_cdf([combined_times['td'].to_list()], 'time difference in seconds', './cdf_of_time_difference_hits_seconds.pdf', islogx=True)

# Time difference analysis for users

In [ ]:
author_td_df = pd.DataFrame()
authors_tds = []

In [ ]:
authors = hits['author'].unique()

In [ ]:
hits.shape

In [ ]:
len(authors)

In [ ]:
import numpy as np
for a in authors:
    tds = combined_times[hits['author']==a]['td'].to_list()
    avg_td = np.mean(tds)
    authors_tds.append({'author': a, 'atd': avg_td})

In [ ]:
len(authors_tds)

In [ ]:
author_td_df = pd.DataFrame(authors_tds)

In [ ]:
plot_cdf([author_td_df['atd'].to_list()], 'time difference in hours', './cdf_of_avg_time_difference_per_author_log.pdf', islogx=False)

# plot cdf of posts per author with time difference 1w, 2w , 1m

In [ ]:
combined_hits = pd.DataFrame()

In [ ]:
hits = pd.read_csv('./results/combined_hits_2017.04.csv')

In [ ]:
combined_hits = combined_hits.append(hits, ignore_index=True)

In [ ]:
hits = combined_hits

In [ ]:
hits = hits.rename(columns={'tweet_time': 'post_time'})
del hits['Unnamed: 0']

In [ ]:
authors = []
for a in hits['author']:
    s = a.split()
    authors.append(s[1])

In [ ]:
del hits['author']
hits['author'] = authors

In [ ]:
times = []
for idx, row in hits.iterrows():
    element = fdata[fdata['tweetid'] == row['tweet_id']]
    time = element['tweet_time'].to_string()
    time = time.split()
    try:
        times.append(time[1]+' '+time[2])
    except IndexError:
        times.append(time[0])

In [ ]:
hits['tweet_time'] = times

In [ ]:
hits = hits[hits['tweet_time'] != 'Series([],']

In [ ]:
hits = hits[hits['post_time'] != 'Series([],']

In [ ]:
t = pd.to_datetime(hits.tweet_time, infer_datetime_format=True)
del hits['tweet_time']
hits['tweet_time'] = t
t = pd.to_datetime(hits.post_time, infer_datetime_format=True)
del hits['post_time']
hits['post_time'] = t

In [ ]:
td = (hits.post_time - hits.tweet_time).astype('timedelta64[h]')
hits['td'] = td

In [ ]:
hits.shape

In [ ]:
hits.head()

In [ ]:
one_week_hits = hits[(hits['td'] >= -168) & (hits['td'] <= 168)]
one_week_hits.shape

In [ ]:
result = one_week_hits.groupby('author').nunique()

In [ ]:
onewresults = result['post_id'].to_list()

In [ ]:
two_week_hits = hits[(hits['td'] >= -336) & (hits['td'] <= 336)]
two_week_hits.shape

In [ ]:
result = two_week_hits.groupby('author').nunique()

In [ ]:
twowresults = result['post_id'].to_list()

In [ ]:
one_month_hits = hits[(hits['td'] >= -744) & (hits['td'] <= 744)]
one_month_hits.shape

In [ ]:
result = one_month_hits.groupby('author').nunique()


In [ ]:
onemresults = result['post_id'].to_list()

In [ ]:
print(len(onewresults))
print(len(twowresults))
print(len(onemresults))

In [ ]:
plot_cdf([onewresults, twowresults, onemresults], 'hits/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './cdf_of_hits_per_author_for_varying_td.pdf', islogx=True)

In [ ]:
plot_cdf([onewresults, twowresults, onemresults], 'tweets/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './cdf_of_tweets_per_author_for_varying_td.pdf', islogx=True)

In [ ]:
plot_cdf([onewresults, twowresults, onemresults], 'posts/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './cdf_of_posts_per_author_for_varying_td.pdf', islogx=True)

In [ ]:
onew = one_week_hits.groupby('author').count()
onew.shape

In [ ]:
owr = onew[onew['td']!=1]['td'].to_list()

In [ ]:
len(owr)

In [ ]:
df1 = onew[onew['td']!=1].index.tolist()
fow = one_week_hits[one_week_hits['author'].isin(df1)]
print(fow.shape)
owr = fow.groupby('author').nunique()

In [ ]:
twow = two_week_hits.groupby('author').count()
twow.shape

In [ ]:
twr = twow[twow['td']!=1]['td'].to_list()
len(twr)

In [ ]:
df1 = twow[twow['td']!=1].index.tolist()
fow = two_week_hits[two_week_hits['author'].isin(df1)]
print(fow.shape)
twr = fow.groupby('author').nunique()

In [ ]:
onem = one_month_hits.groupby('author').count()
onem.shape

In [ ]:
omr = onem[onem['td']!=1]['td'].to_list()
len(omr)

In [ ]:
df1 = onem[onem['td']!=1].index.tolist()
fow = one_month_hits[one_month_hits['author'].isin(df1)]
print(fow.shape)
omr = fow.groupby('author').nunique()

In [ ]:
plot_cdf([owr, twr, omr], 'hits/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './cdf_of_hits_per_author_for_varying_td_filtered.pdf', islogx=True)

In [ ]:
onew = one_week_hits.groupby('author').nunique()
print(onew.shape)
owr = onew['tweet_id'].to_list()
twow = two_week_hits.groupby('author').nunique()
print(twow.shape)
twr = twow['tweet_id'].to_list()
print(len(twr))
onem = one_month_hits.groupby('author').nunique()
print(onem.shape)
omr = onem['tweet_id'].to_list()
print(len(omr))
plot_cdf([owr, twr, omr], 'tweets/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './cdf_of_tweets_per_author_for_varying_td_filtered.pdf', islogx=True)

In [ ]:
plot_cdf([owr['tweet_id'].tolist(), twr['tweet_id'].tolist(), omr['tweet_id'].tolist()], 'tweets/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './cdf_of_tweets_per_author_for_varying_td_filtered.pdf', islogx=True)

In [ ]:
onew = one_week_hits.groupby('author').nunique()
print(onew.shape)
owr = onew['post_id'].to_list()
twow = two_week_hits.groupby('author').nunique()
print(twow.shape)
twr = twow['post_id'].to_list()
print(len(twr))
onem = one_month_hits.groupby('author').nunique()
print(onem.shape)
omr = onem['post_id'].to_list()
print(len(omr))
plot_cdf([owr, twr, omr], 'posts/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './cdf_of_posts_per_author_for_varying_td_filtered.pdf', islogx=True)

In [ ]:
plot_cdf([owr['post_id'].tolist(), twr['post_id'].tolist(), omr['post_id'].tolist()], 'posts/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './cdf_of_posts_per_author_for_varying_td_filtered.pdf', islogx=True)

# Most frequently occuring tweets within one weel time difference

In [ ]:
frequent_tweets = fow.groupby('tweet_id').count()

In [ ]:
frequent_tweets_df = pd.DataFrame()

In [ ]:
frequent_tweets_df['tweet_id'] = frequent_tweets.index.tolist()

In [ ]:
frequent_tweets_df['count'] = frequent_tweets.post_id.tolist()

In [ ]:
frequent_tweets_df.shape

In [ ]:
frequent_tweets_df = frequent_tweets_df.sort_values(by=['count'], ascending=False)

In [ ]:
related_data = data[data['tweetid'].isin(frequent_tweets_df['tweet_id'].tolist())]

In [ ]:
related_data.shape

In [ ]:
texts = []
for i in frequent_tweets_df['tweet_id'].to_list():
    texts.append((related_data[related_data['tweetid'] == i]['tweet_text'].values)[0])

In [ ]:
len(texts)

In [ ]:
frequent_tweets_df['tweet_text'] = texts

In [ ]:
frequent_tweets_df.head(100)

In [ ]:
frequent_tweets_df = frequent_tweets_df.rename(columns={'count': 'frequency'})

In [ ]:
frequent_tweets_df.to_csv('./results/frequent_tweets_one_month_td_for_authors_with_multiple_hits.csv')

In [ ]:
k = pd.DataFrame()
k1 = one_month_hits.groupby(['author'])['post_id'].unique().reset_index(name="unique_post_ids")
k2 = one_month_hits.groupby(['author'])['post_id'].unique().reset_index(name="unique_post_ids")
k3 = one_month_hits.groupby(['author'])['post_id'].unique().reset_index(name="unique_post_ids")
k = k.append(k1, ignore_index = True)
k = k.append(k2, ignore_index = True)
k = k.append(k3, ignore_index = True)
k = k.groupby('author').agg({'unique_post_ids':list}).reset_index()
k.columns

In [ ]:

((k.unique_post_ids.tolist())[0])
k = np.concatenate([((k.unique_post_ids.tolist())[0]), ((k.unique_post_ids.tolist())[0]), ((k.unique_post_ids.tolist())[0])], axis=0)
np.unique(k)

In [ ]:
k = one_month_hits.groupby(['author'])["td"].count().reset_index(name="count")

In [ ]:
k = k.append( one_month_hits.groupby(['author'])["td"].count().reset_index(name="count"), ignore_index = True)

In [ ]:
one_month_hits.groupby(['author'])["td"].mean().reset_index(name="average")

In [ ]:
k.head()

In [ ]:
k.groupby(['author'])['count'].sum().reset_index(name="count")

In [ ]:
import numpy as np 
import pandas as pd
import csv
import os
import re
import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import random
import emoji
import string
import json
import nltk
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
import collections
import glob as glob
import pandas as pd
import numpy as np
from collections import Counter
import json
import tldextract
import re
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from matplotlib.ticker import FuncFormatter
import gensim
from gensim.parsing import PorterStemmer
import networkx as nx
from dateutil.relativedelta import relativedelta
import ast
import warnings
warnings.filterwarnings('ignore')

t_col = "#235dba"
g_col = "#005916"
c_col = "#a50808"
r_col = "#ff9900"
black = "#000000"
pink = "#f442f1"
t_ls = '-'
r_ls = '--'
c_ls = ':'
g_ls = '-.'
colors = [t_col, c_col, g_col, r_col, black, 'c', 'm', pink]
line_styles = [t_ls, r_ls, c_ls, g_ls,t_ls, r_ls, c_ls, g_ls, t_ls]

# list of all the dataset files
dataset_paths = ["../../datasets/russia_052020_tweets_csv_hashed_2.csv", 
         "../../datasets/russian_linked_tweets_csv_hashed.csv", 
         "../../datasets/ira_tweets_csv_hashed.csv", 
         "../../datasets/russia_201906_1_tweets_csv_hashed.csv"]

# path to store the entire combined dataset
combined_dataset_path = "../datasets/russian_trolls.csv"

# returns a pandas dataframe consisting of entries from all the dataset files
def get_combined_dataset(paths):
    data = pd.concat((pd.read_csv(file) for file in tqdm(paths)))
    return data

data = get_combined_dataset(dataset_paths)
print("Number of tweets in the dataset: ", data.shape[0])

def plot_cdf(list_counts, xlabel, path, leg=False, islogx=True, xlimit=False):
    t_col = "#235dba"
    g_col = "#005916"
    c_col = "#a50808"
    r_col = "#ff9900"
    black = "#000000"
    pink = "#f442f1"
    t_ls = '-'
    r_ls = '--'
    c_ls = ':'
    g_ls = '-.'

    markers = [".", "o", "v", "^", "<", ">", "1", "2"]
    colors = [t_col, c_col, g_col, r_col, black, 'c', 'm', pink]
    line_styles = [t_ls, r_ls, c_ls, g_ls,t_ls, r_ls, c_ls, g_ls, t_ls]
    colors = colors[1:]
    line_styles= line_styles[1:]
    while(len(list_counts) > len(colors)):
        colors = colors + shuffle(colors)
        line_styles = line_styles + shuffle(line_styles)
        
    if xlimit:
        l2 = []
        for l in list_counts:
            l2_1 = [x for x in l if x<=xlimit]
            l2.append(l2_1)
        list_counts = l2
    
    for l in list_counts:
        l.sort()
    fig, ax = plt.subplots(figsize=(6,4))
    yvals = []
    for l in list_counts:
        yvals.append(np.arange(len(l))/float(len(l)-1))
    for i in range(len(list_counts)):
        ax.plot(list_counts[i], yvals[i], color=colors[i], linestyle=line_styles[i])
    if islogx:
        ax.set_xscale("log")
    plt.xlabel(xlabel)
    plt.ylabel('CDF')
    plt.grid()
    for item in ([ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(13)
    
    if leg:
        plt.legend(leg, loc='best', fontsize=13)
    
    plt.show()
    fig.savefig(path, bbox_inches='tight')

# Collect all hits above 0.8 cosine similarity
months = ['2016-02','2016-03','2016-04','2016-05','2016-10','2017-01','2017-02','2017-03','2017-04', '2016-12', '2016-06']

def get_relevant_hits(month):
    combined_hits = pd.DataFrame()
    reddit_data = pd.read_csv('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/posts/posts-'+ month +'.csv')
    print('reddit data shape',reddit_data.shape)
    count = 0
    for f in glob.glob('../../reddit_dataset/comments/scores/RC_'+ month +'.bz2.decompressed/*.txt'):
        hits = []
        with open(f , 'r') as content_file:
            content = content_file.read()
            json_data = content.replace('][',',')
            j_object = json.loads(json_data)
            json_df = pd.DataFrame(j_object)
            j_df = json_df[(json_df['cosine_similarity'] > 0.8)]
            fdata = data[data['tweetid'].isin(j_df['tweet_id'].to_list())]
            fposts = reddit_data[reddit_data['id'].isin(j_df['post_id'].to_list())]
            print('fposts  shape', fposts.shape)
            for index, row in j_df.iterrows():
                post = fposts[fposts['id'] == row['post_id']]
                if not (post.empty):
                    try:
                        tweet = fdata[fdata['tweetid'] == row['tweet_id']]
                        post_time = (post['created_utc'].values)[0]
                        post_time = (datetime.datetime.fromtimestamp(post_time))
                        element = fdata[fdata['tweetid'] == row['tweet_id']]
                        tweet_time = (element['tweet_time'].values)[0]
                        hits.append({'tweet_id': row['tweet_id'], 'post_id': row['post_id'], 
                                     'post_author': (post['author'].values)[0],
                                     'post_time': post_time, 'tweet_time': tweet_time})
                    except IndexError:
#                         print(post)
                        print(post['created_utc'])
                    except ValueError:
                        print('Its a value error')
                        print(post['created_utc'])
                        print(post_time)
                    except Exception as ex:
                        print('createdutc: ', post['created_utc'])
                        print(ex)
                        post_times = post_time.split()
                        post_time = post_times[1]
                        
            hits = pd.DataFrame(hits)
#             print('collected hits: ', hits.shape)
            if not (hits.empty):
                t = pd.to_datetime(hits.tweet_time, infer_datetime_format = True)
                del hits['tweet_time']
                hits['tweet_time'] = t
                hits['td'] = (hits.post_time - hits.tweet_time).astype('timedelta64[h]')
                hits = hits[(hits['td'] >= -672) & (hits['td'] <= 672)]
#             print('After filtering: ', hits.shape)
        combined_hits = combined_hits.append(hits, ignore_index = True)
        count = count + 1
        print(count)
        print(combined_hits.shape)
    return combined_hits

def save_data(name, data):
    data.to_csv('./results/group_by_author/'+name+'.csv')
    
def get_unique_elements_count(data, field):
    counts = []
    author_wise_lists = data[field].tolist()
    for author_posts in author_wise_lists():
        concatenated_list = []
        for posts in author_posts:
            concatenated_list = np.concatenate([concatenated_list, posts], axis=0)
        unique_elements = np.unique(concatenated_list)
        counts.append(len(unique_elements))
    return counts

def get_count(data, field):
    counts = []
    author_wise_lists = data[field].tolist()
    for author_posts in author_wise_lists:
        counts.append(len(author_posts))
    return counts
   
authors_avg_time_difference = pd.DataFrame()
ow_hits_per_author = pd.DataFrame()
ow_posts_per_author = pd.DataFrame()
ow_tweets_per_author = pd.DataFrame()

tw_hits_per_author = pd.DataFrame()
tw_posts_per_author = pd.DataFrame()
tw_tweets_per_author = pd.DataFrame()

om_hits_per_author = pd.DataFrame()
om_posts_per_author = pd.DataFrame()
om_tweets_per_author = pd.DataFrame()

# different approach
complete_hits = pd.DataFrame()
for m in months:
    relevant_hits = pd.read_csv('./results/hits/'+m+'.csv')
    complete_hits = complete_hits.append(relevant_hits, ignore_index = True)
    print('Number of hits in month ', m, 'with time difference less than a month :', (relevant_hits.shape)[0])

owh = complete_hits[(complete_hits['td'] >= -168) & (complete_hits['td'] <= 168)]
hits_per_author = owh.groupby(['post_author'])["post_id"].count().reset_index(name="count")
ow_hits_per_author = ow_hits_per_author.append(hits_per_author, ignore_index = True)
avg_td = owh.groupby(['post_author'])["td"].mean().reset_index(name="average")
authors_avg_time_difference = authors_avg_time_difference.append(avg_td, ignore_index = True)
posts_per_author = owh.groupby(['post_author'])["post_id"].unique().reset_index(name="unique_post_ids")
ow_posts_per_author = ow_posts_per_author.append(posts_per_author, ignore_index = True)
tweets_per_author = owh.groupby(['post_author'])["tweet_id"].unique().reset_index(name="unique_tweet_ids")
ow_tweets_per_author = ow_tweets_per_author.append(tweets_per_author, ignore_index = True)

twh = complete_hits[(complete_hits['td'] >= -336) & (complete_hits['td'] <= 336)]
hits_per_author = twh.groupby(['post_author'])["post_id"].count().reset_index(name="count")
tw_hits_per_author = tw_hits_per_author.append(hits_per_author, ignore_index = True)
posts_per_author = twh.groupby(['post_author'])["post_id"].unique().reset_index(name="unique_post_ids")
tw_posts_per_author = tw_posts_per_author.append(posts_per_author, ignore_index = True)
tweets_per_author = twh.groupby(['post_author'])["tweet_id"].unique().reset_index(name="unique_tweet_ids")
tw_tweets_per_author = tw_tweets_per_author.append(tweets_per_author, ignore_index = True)

hits_per_author = complete_hits.groupby(['post_author'])["post_id"].count().reset_index(name="count")
om_hits_per_author = om_hits_per_author.append(hits_per_author, ignore_index = True)
posts_per_author = complete_hits.groupby(['post_author'])["post_id"].unique().reset_index(name="unique_post_ids")
om_posts_per_author = om_posts_per_author.append(posts_per_author, ignore_index = True)
tweets_per_author = complete_hits.groupby(['post_author'])["tweet_id"].unique().reset_index(name="unique_tweet_ids")
om_tweets_per_author = om_tweets_per_author.append(tweets_per_author, ignore_index = True)

plot_cdf([authors_avg_time_difference.average.tolist()], 'average time difference',leg=['within 1 week time difference'], path = './results/group_by_author/cdf_of_avgtd_per_author_for_one_week_td.pdf', islogx=False)

plot_cdf([ow_hits_per_author['count'].tolist(), tw_hits_per_author['count'].tolist(), om_hits_per_author['count'].tolist()], 'hits/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './results/group_by_author/cdf_of_hits_per_author_for_varying_td.pdf', islogx=True)

ow_posts = get_count(ow_posts_per_author, 'unique_post_ids')
tw_posts = get_count(tw_posts_per_author, 'unique_post_ids')
om_posts = get_count(om_posts_per_author, 'unique_post_ids')

plot_cdf([ow_posts, tw_posts, om_posts], 'posts/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './results/group_by_author/cdf_of_posts_per_author_for_varying_td.pdf', islogx=True)

ow_tweets = get_count(ow_tweets_per_author, 'unique_tweet_ids')
tw_tweets = get_count(tw_tweets_per_author, 'unique_tweet_ids')
om_tweets = get_count(om_tweets_per_author, 'unique_tweet_ids')

plot_cdf([ow_tweets, tw_tweets, om_tweets], 'tweets/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './results/group_by_author/cdf_of_tweets_per_author_for_varying_td.pdf', islogx=True)

In [ ]:
t_col = "#235dba"
g_col = "#005916"
c_col = "#a50808"
r_col = "#ff9900"
black = "#000000"
pink = "#f442f1"
t_ls = '-'
r_ls = '--'
c_ls = ':'
g_ls = '-.'
colors = [t_col, c_col, g_col, r_col, black, 'c', 'm', pink]
line_styles = [t_ls, r_ls, c_ls, g_ls,t_ls, r_ls, c_ls, g_ls, t_ls]
plot_cdf([authors_avg_time_difference.average.tolist()], 'average time difference',leg=['within 1 week time difference'], path = './results/group_by_author/cdf_of_avgtd_per_author_for_one_week_td.pdf', islogx=False)

plot_cdf([ow_hits_per_author['count'].tolist(), tw_hits_per_author['count'].tolist(), om_hits_per_author['count'].tolist()], 'hits/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './results/group_by_author/cdf_of_hits_per_author_for_varying_td.pdf', islogx=True)

ow_posts = get_count(ow_posts_per_author, 'unique_post_ids')
tw_posts = get_count(tw_posts_per_author, 'unique_post_ids')
om_posts = get_count(om_posts_per_author, 'unique_post_ids')

plot_cdf([ow_posts, tw_posts, om_posts], 'posts/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './results/group_by_author/cdf_of_posts_per_author_for_varying_td.pdf', islogx=True)

ow_tweets = get_count(ow_tweets_per_author, 'unique_tweet_ids')
tw_tweets = get_count(tw_tweets_per_author, 'unique_tweet_ids')
om_tweets = get_count(om_tweets_per_author, 'unique_tweet_ids')

plot_cdf([ow_tweets, tw_tweets, om_tweets], 'tweets/author',leg=['1 week time difference', '2 week time difference', '1 month time difference'], path = './results/group_by_author/cdf_of_tweets_per_author_for_varying_td.pdf', islogx=True)

# Extracting, resolving and analysing URLs in tweet index

In [ ]:
import pandas as pd
index = pd.read_csv('./relevant_tweets.csv')
index_tweet_ids = index['tweetid'].tolist()
index_data = data[data['tweetid'].isin(index_tweet_ids)]
index_data = index_data.drop_duplicates(subset='tweetid', keep="last")

In [ ]:
def extract_urls(row):
    links = re.findall("(?P<url>https?://[^\s]+)", str(row['tweet_text']))
    return links

def has_links(row):
    links = re.findall("(?P<url>https?://[^\s]+)", str(row['tweet_text']))
    if len(links)==0:
        return False
    else:
        return True
    
index_data['has_urls'] = index_data.apply(has_links, axis=1)
tweets_with_urls = index_data[index_data['has_urls'] == True]
tweets_with_urls['urls'] = tweets_with_urls.apply(extract_urls, axis=1)

In [ ]:
all_urls = []
for l in tweets_with_urls['urls'].tolist():
    all_urls = all_urls + l

In [ ]:
import requests
count = 0
unresolved_urls = []
resolved_urls = []
for url in all_urls:
    try:
        resp = requests.head(url)
        resolved_url = resp.headers["Location"]
        resolved_urls.append(resolved_url)
        print(resolved_url)
    except:
        count = count + 1
        print(count)
        unresolved_urls.append(url)

In [ ]:
url_df = pd.DataFrame()
url_df['url'] = resolved_urls

In [ ]:
# Finding information on the domain for each URL

# Extracting the domain from the URL
domain_re = re.compile("http[s]*://([^/]+)")
url_df["domain"] = url_df.url.apply(lambda X: domain_re.search(X).group(1))

# Removing leading www.
www_re = re.compile("^www\.")
url_df["domain"] = url_df["domain"].apply(lambda X: www_re.sub("", X))


In [ ]:
url_counts = url_df['domain'].value_counts()
url_counts = url_df.groupby(['domain'])["url"].count().reset_index(name="count")
url_counts = url_counts.sort_values('count', ascending=False)
url_df.to_csv('./results/url_analysis/urls_domains.csv')
url_counts.to_csv('./results/url_analysis/domain_counts.csv')

# Get URL title and compare with tweet text


In [ ]:
try:
        resp = requests.head(url)
        resolved_url = resp.headers["Location"]
        resolved_urls.append(resolved_url)
        print(resolved_url)
    except:
        count = count + 1
        print(count)
        unresolved_urls.append(url)

In [ ]:
tweets_with_urls.columns

In [ ]:
allowed_schemes = ('http', 'ftp')
allowed_exts = ('png', 'jpg')
from urllib.parse import urlparse
url = urlparse(" https://en.insidesyriamc.com/2016/11/03/summary-of-armed-clashes-the-background-of-crisis")
print(url.scheme)
url.scheme in allowed_schemes
url.path.rsplit('.', 1)[1] in allowed_exts
print(url.path.rsplit('/'))

In [ ]:
tweet_text_url_titles = []

In [ ]:
import string
from urllib.parse import urlparse
count = 0
for index,row in tweets_with_urls.iterrows():
    print(count)
    count = count + 1
    urls= row['urls']
    for url in urls:
        try:
            resp = requests.head(url)
            resolved_url = resp.headers["Location"]
            try:
                if "https://twitter.com" not in resolved_url:
                    u = urlparse(resolved_url)
                    if u.netloc in ["gopthedailydose.com","en.insidesyriamc.com"]:
                        u_parts = u.path.rsplit('/')
                        title = u_parts[4].replace("-", " ")
                        tweet_text_url_titles.append({'tweetid': row['tweetid'],
                                                         'tweet_text': row['tweet_text'],
                                                         'url': resolved_url,
                                                         'page_title': title})
                        print('title from page: ', title)
                        print('tweet text: ', row['tweet_text'])
                    else:
                        soup = BeautifulSoup(urllib.urlopen(resolved_url,timeout = 10))
                        if soup.title != None:
                            tweet_text_url_titles.append({'tweetid': row['tweetid'],
                                                         'tweet_text': row['tweet_text'],
                                                         'url': resolved_url,
                                                         'page_title': soup.title.string})
                            print('title from page: ', soup.title.string)
                            print('tweet text: ', row['tweet_text'])
            except Exception as e: 
                print('url when error: ',resolved_url)
                print('tweet text: ', row['tweet_text'])
                print('exception: ', e)
        except Exception as e:
            print('Error resolving: ', e)
            

In [ ]:
titles_text = pd.DataFrame(tweet_text_url_titles)

In [ ]:
titles_text.to_csv('./results/url_new_titles.csv')

In [ ]:
tweets_removed_news = index_data[~index_data['tweetid'].isin(titles_text['tweetid'].tolist())]

In [ ]:
index_data.shape

In [ ]:
tweets_removed_news.shape

# Repeat for more urls

In [ ]:
tweets_with_urls2 = tweets_with_urls[~tweets_with_urls['tweetid'].isin(titles_text['tweetid'].tolist())]

In [ ]:
tweet_text_url_titles2 = []
import string
from urllib.parse import urlparse
count = 0
for index,row in tweets_with_urls2.iterrows():
    print(count)
    count = count + 1
    urls= row['urls']
    for url in urls:
        try:
            resp = requests.head(url)
            resolved_url = resp.headers["Location"]
            tweet_text_url_titles2.append({'tweetid': row['tweetid'],
                                                         'tweet_text': row['tweet_text'],
                                                         'url': resolved_url})
        except Exception as e:
            print('Error resolving: ', e)
            

In [ ]:
d = pd.DataFrame(tweet_text_url_titles2)
d.to_csv('./results/for_annotation_url_titles.csv')

## Putting all the url part together

In [ ]:
annotated_urls = pd.read_csv('./results/annotated_url_titles.csv')

In [ ]:
annotated_urls.shape

In [ ]:
d['is_matching'] = annotated_urls['is_matching'].tolist()

In [ ]:
ids = d[(d['is_matching'] == 1) | (d['is_matching'] == 11)]['tweetid'].tolist()

In [ ]:
len(ids)

In [ ]:
tweets_that_ar_not_page_titles = tweets_removed_news[~tweets_removed_news['tweetid'].isin(ids)]

In [ ]:
tweets_that_ar_not_page_titles.shape

In [ ]:
tweets_removed_news.to_csv('./results/index_tweets_after_removing_news_title_tweets.csv')

In [ ]:
ids = ids + titles_text['tweetid'].tolist()

In [ ]:
ids = list(set(ids))

In [ ]:
len(ids)

In [ ]:
ids_df = pd.DataFrame()
ids_df['tweetid'] = ids

In [ ]:
ids_df.to_csv('./results/tweet_ids_of_news_titles_tweets.csv')

# Get popular authors

In [ ]:
months = ['2016-01','2016-02','2016-03','2016-04','2016-05','2016-06',
         '2016-07','2016-08','2016-09','2016-10','2016-11','2016-12',
         '2017-01','2017-02','2017-03','2017-04','2017-05','2017-06',]
complete_hits = pd.DataFrame()
for m in months:
    relevant_hits = pd.read_csv('./results/relevent_hits/'+m+'.csv')
    complete_hits = complete_hits.append(relevant_hits, ignore_index = True)
    print('Number of hits in month ', m, 'with time difference less than a month :', (relevant_hits.shape)[0])

In [ ]:
del complete_hits['Unnamed: 0']
del complete_hits['bot_author']

In [ ]:
title_tweetids_df = pd.read_csv('./results/tweet_ids_of_news_titles_tweets.csv')
title_tweetids = title_tweetids_df['tweetid'].tolist()
print('complete hits before removing news titles: ', complete_hits.shape)
complete_hits_without_news = complete_hits[~complete_hits['tweet_id'].isin(title_tweetids)]
print('complete hits before after news titles: ', complete_hits_without_news.shape)
# Remove authors with name as bot or auto
def is_bot_auto(row):
    author = str(row['post_author'])
    if 'bot' in author:
        return True
    elif 'auto' in author:
        return True
    else:
        return False
complete_hits_without_news['is_bot'] = complete_hits_without_news.apply(is_bot_auto, axis=1)
complete_hits = complete_hits_without_news[~complete_hits_without_news['is_bot']==True]
print('complete hits after removing bot authors: ', complete_hits.shape)

In [ ]:
hits_per_author = complete_hits.groupby(['post_author'])["post_id"].count().reset_index(name="count")
hits_per_author = hits_per_author.sort_values(by='count', ascending=False)
hits_per_author = hits_per_author[hits_per_author['post_author'] != '[deleted]']
popular_1000 = hits_per_author.head(1000)

In [ ]:
hits_per_author.shape

In [ ]:
author_status = []
count = 0
for author in popular_1000['post_author'].tolist():
    
    url = 'https://www.reddit.com/user/'+ author +'.json'
    try:
        resp = requests.get(url,headers = {'User-agent': 'your bot 0.1'})
        author_status.append(resp.status_code)
        count = count + 1
        print('count', count)
    except Exception as e:
        print(e)
        print(author)
        author_status.append(None)
    

In [ ]:
popular_1000['author_status'] = author_status

In [ ]:
pd.set_option('display.max_rows', 500)
suspended_authors = popular_1000[popular_1000['author_status']==403]

In [ ]:
authors_404 = popular_1000[popular_1000['author_status']==404]

# check if author is alive

In [ ]:
import requests
count = 0
url = 'https://www.reddit.com/user/samacharbot2.json'
try:
    resp = requests.get(url,headers = {'User-agent': 'your bot 0.1'})
except Exception as e:
    print(e)
    count = count + 1
    print('count', count)


In [ ]:
resp.status_code

In [ ]:
resp.json()

# Popular subreddits

In [ ]:
subreddits = complete_hits.groupby(['subreddit'])["post_id"].count().reset_index(name="count")

In [ ]:
subreddits = subreddits.sort_values(by='count', ascending=False)
ps_100 = subreddits.head(100)

# Rank authors by time difference

In [ ]:
authors_with_atleast_5_hits = pd.DataFrame()
authors_with_atleast_5_hits = authors_with_atleast_5_hits.append(ow_hits_per_author[ow_hits_per_author['count'] >= 5], ignore_index = True)
authors_with_atleast_5_hits = authors_with_atleast_5_hits.append(tw_hits_per_author[tw_hits_per_author['count'] >= 5], ignore_index = True)
authors_with_atleast_5_hits = authors_with_atleast_5_hits.append(om_hits_per_author[om_hits_per_author['count'] >= 5], ignore_index = True)
authors_with_atleast_5_hits = authors_with_atleast_5_hits.drop_duplicates(subset='post_author', keep="last")

In [ ]:
count = 0
avg_tds = []
mod_avg_tds = []
for author in authors_with_atleast_5_hits['post_author'].tolist():
    r_hits = complete_hits[complete_hits['post_author'] == author]
    avg_tds.append(np.mean(r_hits['td'].tolist()))
    mod_avg_tds.append(np.mean(np.abs(r_hits['td'].tolist())))
    count = count + 1
    print(count)

In [ ]:
authors_with_atleast_5_hits['avg_td'] = avg_tds
authors_with_atleast_5_hits['avg_abs_td'] = mod_avg_tds
authors_with_atleast_5_hits = authors_with_atleast_5_hits.sort_values('avg_abs_td')
authors_with_atleast_5_hits = authors_with_atleast_5_hit.rename(columns={'count': 'number_of_hits'})

In [ ]:
authors_with_atleast_5_hits.to_csv('./results/group_by_author/rank_by_td.csv')

# Get all hits of the top 5 authors and also their avg td

In [ ]:
top_5_authors = []

In [ ]:
om_hits_per_author = om_hits_per_author.sort_values('count', ascending=False)

In [ ]:
om_hits_per_author = popular_1000

In [ ]:
for a in om_hits_per_author['post_author'].tolist()[0:6]:
    if a != '[deleted]':
        print(a)
        top_5_authors.append(a)

In [ ]:
top_5_authors = list(set(top_5_authors))

In [ ]:
hits_top_5_authors = pd.DataFrame()
for author in top_5_authors:
    r_hits = complete_hits[complete_hits['post_author'] == author]
    hits_top_5_authors = hits_top_5_authors.append(r_hits, ignore_index = True)

In [ ]:
hits_top_5_authors.head()

In [ ]:
avg_td_top_5_author = hits_top_5_authors.groupby(['post_author'])["td"].mean().reset_index(name="average_td")

In [ ]:
relevant_posts = pd.DataFrame()

In [ ]:
for m in months:
    reddit_data = pd.read_csv('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/posts/posts-' + m +'.csv')
    fposts = reddit_data[reddit_data['id'].isin(hits_top_5_authors['post_id'].to_list())]
    relevant_posts = relevant_posts.append(fposts)

In [ ]:
twitter_data = pd.read_csv('./relevant_tweets.csv')

In [ ]:
relevant_posts.head()

In [ ]:
def get_posts(row):
    post_id = row['post_id']
    post = relevant_posts[relevant_posts['id'] == post_id]
    return post['body'].values

In [ ]:
def get_subreddits(row):
    post_id = row['post_id']
    post = relevant_posts[relevant_posts['id'] == post_id]
    return post['subreddit'].values

In [ ]:
def get_tweets(row):
    tweet_id = row['tweet_id']
    tweet = twitter_data[twitter_data['tweetid'] == tweet_id]
    return tweet['tweet_text'].values

In [ ]:
hits_top_5_authors.head(20)

In [ ]:
hits_top_5_authors['tweet_text'] = hits_top_5_authors.apply(get_tweets, axis=1)

In [ ]:
hits_top_5_authors = hits_top_5_authors.drop_duplicates(subset=['post_id','tweet_id'] , keep="last")

In [ ]:
hits_top_5_authors.shape

# Compare the percentage of suspension based on time difference

In [ ]:
months = ['2016-01','2016-02','2016-03','2016-04','2016-05','2016-06',
         '2016-07','2016-08','2016-09','2016-10','2016-11','2016-12',
         '2017-01','2017-02','2017-03','2017-04','2017-05','2017-06']

In [28]:
import pandas as pd
complete_hits = pd.DataFrame()
for m in months:
    relevant_hits = pd.read_csv('./results/relevent_hits/'+m+'.csv')
    complete_hits = complete_hits.append(relevant_hits, ignore_index = True)
    print('Number of hits in month ', m, 'with time difference less than a month :', (relevant_hits.shape)[0])
del complete_hits['Unnamed: 0']
del complete_hits['bot_author']
print(complete_hits.shape)
complete_hits = complete_hits.drop_duplicates(subset=['tweet_id','post_id'], keep='last')
print(complete_hits.shape)

Number of hits in month  2016-01 with time difference less than a month : 16553
Number of hits in month  2016-02 with time difference less than a month : 41170
Number of hits in month  2016-03 with time difference less than a month : 51063
Number of hits in month  2016-04 with time difference less than a month : 33508
Number of hits in month  2016-05 with time difference less than a month : 37426
Number of hits in month  2016-06 with time difference less than a month : 33341
Number of hits in month  2016-07 with time difference less than a month : 42182
Number of hits in month  2016-08 with time difference less than a month : 27679
Number of hits in month  2016-09 with time difference less than a month : 33002
Number of hits in month  2016-10 with time difference less than a month : 39516
Number of hits in month  2016-11 with time difference less than a month : 48811
Number of hits in month  2016-12 with time difference less than a month : 23263
Number of hits in month  2017-01 with ti

In [ ]:
complete_hits_1y = pd.DataFrame()
for m in months:
    relevant_hits = pd.read_csv('./results/one_year/'+m+'.csv')
    complete_hits_1y = complete_hits_1y.append(relevant_hits, ignore_index = True)
    print('Number of hits in month ', m, 'with time difference less than a year :', (relevant_hits.shape)[0])
del complete_hits_1y['Unnamed: 0']
del complete_hits_1y['bot_author']

In [29]:
title_tweetids_df = pd.read_csv('./results/tweet_ids_of_news_titles_tweets.csv')
title_tweetids = title_tweetids_df['tweetid'].tolist()
print('complete hits before removing news titles: ', complete_hits.shape)
complete_hits_without_news = complete_hits[~complete_hits['tweet_id'].isin(title_tweetids)]
print('complete hits before after news titles: ', complete_hits_without_news.shape)
# Remove authors with name as bot or auto
def is_bot_auto(row):
    author = str(row['post_author'])
    if 'bot' in author:
        return True
    elif 'auto' in author:
        return True
    else:
        return False
complete_hits_without_news['is_bot'] = complete_hits_without_news.apply(is_bot_auto, axis=1)
complete_hits = complete_hits_without_news[~complete_hits_without_news['is_bot']==True]
print('complete hits after removing bot authors: ', complete_hits.shape)

hits_per_author = complete_hits.groupby(['post_author'])["post_id"].count().reset_index(name="count")
hits_per_author = hits_per_author.sort_values(by='count', ascending=False)
hits_per_author = hits_per_author[hits_per_author['post_author'] != '[deleted]']


complete hits before removing news titles:  (607528, 9)
complete hits before after news titles:  (515508, 9)


/INET/state-trolls/work/state-trolls/miniconda3/envs/env3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


complete hits after removing bot authors:  (513666, 10)


In [ ]:
print('complete hits before removing news titles: ', complete_hits_1y.shape)
complete_hits_without_news_1y = complete_hits_1y[~complete_hits_1y['tweet_id'].isin(title_tweetids)]
print('complete hits before after news titles: ', complete_hits_without_news_1y.shape)
complete_hits_without_news_1y['is_bot'] = complete_hits_without_news_1y.apply(is_bot_auto, axis=1)
complete_hits_1y = complete_hits_without_news_1y[~complete_hits_without_news_1y['is_bot']==True]
print('complete hits after removing bot authors: ', complete_hits_1y.shape)

hits_per_author_1y = complete_hits_1y.groupby(['post_author'])["post_id"].count().reset_index(name="count")
hits_per_author_1y = hits_per_author_1y.sort_values(by='count', ascending=False)
hits_per_author_1y = hits_per_author_1y[hits_per_author_1y['post_author'] != '[deleted]']

In [30]:
complete_hits['abs_td'] = complete_hits['td'].abs()
avg_td = complete_hits.groupby(['post_author'])["abs_td"].mean().reset_index(name="average")
avg_td_more_hits = avg_td[avg_td['post_author']
                          .isin(hits_per_author[hits_per_author['count']>5]['post_author'].to_list())]
avg_td_2w = avg_td_more_hits[(avg_td_more_hits['average'] >-1209600) & (avg_td_more_hits['average'] < 1209600)]
avg_td_1w = avg_td_more_hits[(avg_td_more_hits['average'] >-604800) & (avg_td_more_hits['average'] < 604800)]

/INET/state-trolls/work/state-trolls/miniconda3/envs/env3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
# How many author with td 2w and hits >5 are suspended?
import glob
author_status = pd.DataFrame()
for f in glob.glob('./results/author_status/*.csv'):
    author_status = author_status.append(pd.read_csv(f),ignore_index=True)

In [ ]:
author_status_1w_more_hits = author_status[author_status['post_author']
                                           .isin(avg_td_1w['post_author'].tolist())]
print(author_status_1w_more_hits[author_status_1w_more_hits['author_status'] == 403].shape)
print(author_status_1w_more_hits[author_status_1w_more_hits['author_status'] == 404].shape)
print(author_status_1w_more_hits[author_status_1w_more_hits['author_status'] == 200].shape)
print(author_status_1w_more_hits[author_status_1w_more_hits['author_status'] == 503])
print(author_status_1w_more_hits[author_status_1w_more_hits['author_status'] == 504])
author_status_1w_more_hits['author_status'].unique()

In [ ]:
author_status_1w_more_hits.shape

In [ ]:
complete_hits_1y['abs_td'] = complete_hits_1y['td'].abs()
avg_td_1y = complete_hits_1y.groupby(['post_author'])["abs_td"].mean().reset_index(name="average")
avg_td_1y_more_hits = avg_td_1y[avg_td_1y['post_author']
                                .isin(hits_per_author_1y[hits_per_author_1y['count']>5]['post_author'].to_list())]

In [ ]:
# How many author with td 1 year and hits >5 are suspended?
import glob
author_status_1y = pd.DataFrame()
for f in glob.glob('./results/one_year/author_status/*.csv'):
    author_status_1y = author_status_1y.append(pd.read_csv(f),ignore_index=True)
author_status_1y = author_status_1y.drop_duplicates(subset=['post_author'], keep='first')
author_status_1y.shape

In [ ]:
author_status_1y_more_hits = author_status_1y[author_status_1y['post_author']
                                           .isin(avg_td_1y_more_hits['post_author'].tolist())]


In [ ]:
author_status_1y_more_hits = author_status_1y_more_hits[~author_status_1y_more_hits['post_author'].isin(author_status_1w_more_hits['post_author'].tolist())]

In [ ]:
author_status_1y_more_hits.shape

In [ ]:
print(author_status_1y_more_hits[author_status_1y_more_hits['author_status'] == 403].shape)
print(author_status_1y_more_hits[author_status_1y_more_hits['author_status'] == 404].shape)
print(author_status_1y_more_hits[author_status_1y_more_hits['author_status'] == 200].shape)
print(author_status_1y_more_hits[author_status_1y_more_hits['author_status'] == 503])
print(author_status_1y_more_hits[author_status_1y_more_hits['author_status'] == 504])
author_status_1y_more_hits['author_status'].unique()

In [ ]:
random_samples = avg_td_1y.sample(n=30000)
random_samples = random_samples[~random_samples['post_author'].isin(author_status_1w_more_hits['post_author'].tolist())]
random_samples_author_status = author_status_1y[author_status_1y['post_author']
                                           .isin(random_samples['post_author'].tolist())]
print(random_samples_author_status.shape)
random_samples_1000 = random_samples_author_status.sample(n=1000)

In [ ]:
print(random_samples_1000[random_samples_1000['author_status'] == 403].shape)
print(random_samples_1000[random_samples_1000['author_status'] == 404].shape)
print(random_samples_1000[random_samples_1000['author_status'] == 200].shape)
print(random_samples_1000[random_samples_1000['author_status'] == 503])
print(random_samples_1000[random_samples_1000['author_status'] == 504])
random_samples_1000['author_status'].unique()

## Get all hits of top 5 authors ranked on td along with distinguished and sim

In [31]:
avg_td_1w['abs_td'] = avg_td_1w['average'].abs()
avg_td_1w = avg_td_1w.sort_values('abs_td', ascending=True)
top_1000_td = avg_td_1w#.head(1000)


/INET/state-trolls/work/state-trolls/miniconda3/envs/env3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
del avg_td_1w['abs_td']

In [33]:
status = []
for idx, row in top_1000_td.iterrows():
    a = row['post_author']
    h = author_status[author_status['post_author']==a]
    status.append((h['author_status'].values)[0])
#     print((h['count'].values)[0])
top_1000_td['author_status'] = status  

In [34]:
counts = []
for idx, row in top_1000_td.iterrows():
    a = row['post_author']
    h = hits_per_author[hits_per_author['post_author']==a]
    counts.append((h['count'].values)[0])
#     print((h['count'].values)[0])
top_1000_td['counts'] = counts  
top_1000_td = top_1000_td.sort_values('counts', ascending=False)

In [38]:
top_1000_td = top_1000_td.sort_values('average', ascending=True)
t10 = top_1000_td.head(100)
t10 = t10.sort_values('counts', ascending=False)
pd.set_option('display.max_rows', 500)

t10.head(100)

,post_author,average,author_status,counts
162434,m0rpheuz,16890.526316,404,38
37571,GXGOW,17598.517241,200,29
204185,yaqooberz,31707.000000,200,24
121860,bernbright-net,25739.000000,200,20
78138,PressXtoHuehue,29400.368421,404,19
100064,TheRA51,18189.000000,200,16
116450,aknifeguy,11580.000000,404,16
153444,jeevs193,12218.500000,200,16
111356,Xenoanthropus,14263.125000,200,16
101249,The_Silver_Avenger,26209.200000,200,15


In [42]:
posts_per_author = complete_hits.groupby(['post_author'])["post_id"].nunique().reset_index(name="count")
posts_per_author = posts_per_author[posts_per_author['post_author'] != '[deleted]']

avg_td_more_posts = avg_td[avg_td['post_author']
                          .isin(posts_per_author[posts_per_author['count']>5]['post_author'].to_list())]
avg_td_posts_1w = avg_td_more_posts[(avg_td_more_posts['average'] >-604800) & (avg_td_more_posts['average'] < 604800)]
print(avg_td_posts_1w.shape)

(239, 2)


In [44]:
avg_td_posts_1w['abs_td'] = avg_td_posts_1w['average'].abs()
top_1000_td_posts = avg_td_posts_1w#.head(1000)

del avg_td_posts_1w['abs_td']
status = []
for idx, row in top_1000_td_posts.iterrows():
    a = row['post_author']
    h = author_status[author_status['post_author']==a]
    status.append((h['author_status'].values)[0])
#     print((h['count'].values)[0])
top_1000_td_posts['author_status'] = status  

/INET/state-trolls/work/state-trolls/miniconda3/envs/env3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/INET/state-trolls/work/state-trolls/miniconda3/envs/env3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [45]:
counts = []
for idx, row in top_1000_td_posts.iterrows():
    a = row['post_author']
    h = posts_per_author[posts_per_author['post_author']==a]
    counts.append((h['count'].values)[0])
#     print((h['count'].values)[0])
top_1000_td_posts['counts'] = counts  
top_1000_td_posts = top_1000_td_posts.sort_values('counts', ascending=False)

/INET/state-trolls/work/state-trolls/miniconda3/envs/env3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [47]:
top_1000_td_posts = top_1000_td_posts.sort_values('average', ascending=True)
t10 = top_1000_td_posts.head(100)
t10 = t10.sort_values('counts', ascending=False)
pd.set_option('display.max_rows', 500)

t10.head(100)

,post_author,average,author_status,counts
172972,optimalg,60782.548387,200,61
88272,Shanti_Ananda,202448.867925,200,51
147135,helpmeredditimbored,338209.645161,200,42
168998,nanami-773,327931.318584,200,40
115969,afterpoop,337003.522727,200,33
103932,Trenchguard,151414.766667,200,30
192496,tgjer,216783.240000,200,25
171010,nosecohn,430833.560000,200,23
202017,wil_daven_,416340.170732,200,23
84975,SFCHCP123,252481.714286,200,21


In [ ]:
top_1000 = top_1000_td['post_author'].head(1000).tolist()
hits = complete_hits[complete_hits['post_author'].isin(top_1000)]
post_ids = hits['post_id'].unique() 

In [ ]:
print(top_1000_td[top_1000_td['author_status']==403].shape)
print(top_1000_td[top_1000_td['author_status']==404].shape)
print(top_1000_td[top_1000_td['author_status']==200].shape)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

y = [10, 3,4.2 , 4.57, 4.9,5]
z = [20, 21,14.4 , 13.71, 14.4, 15.42]
x = [1,2,3,4,5,6]

# for xe, ye in zip(x, y):
#     plt.scatter([xe] * len(ye), ye)

plt.xticks([1, 2, 3, 4, 5 ,6])
plt.axes().set_xticklabels(['top_10', 'top_100', 'top_500', 'top_700', 'top_1000', 'top_1400'])
line = plt.plot(x, y, label = '% of suspension')
# line.set_label('% of suspension')
line2 = plt.plot(x, z, label = '% of deletion')
# line2.set_label('% of deletion')

plt.grid()
plt.legend()
plt.xlabel("Top n authors")
plt.ylabel("% of suspension / deletion")
plt.savefig('./results/corelation.png')

In [ ]:
top_1000_td.to_csv('./results/author_status/top_100_authors_status.csv')
top_1000_td.head(100)

In [ ]:
# related_posts = pd.DataFrame()
for posts in glob.glob('../../reddit_dataset/comments/posts/posts-201*.csv'):
    d = pd.read_csv(posts)
    print(d.columns)
    d = d[d['id'].isin(post_ids)]
    related_posts = related_posts.append(d, ignore_index = True)
    print('finished a file')
print(related_posts.shape)

In [ ]:
distinguished = []
for index,row in hits.iterrows():
    post = related_posts[related_posts['id']==row['post_id']]
    dis = (post['distinguished'].values)[0]
#     print(dis)
    distinguished.append(dis)
hits['distinguished'] = distinguished    
hits = hits[hits['distinguished']!='moderator']
# hits.to_csv('./results/hits_od_top10_authors_ranked_by_td_hits.csv')

In [ ]:
hits_10 = hits

In [ ]:
hits_100 = hits

In [ ]:
hits_1000 = hits

In [ ]:
hits.columns

In [ ]:
plot_cdf([hits_10['abs_td'].tolist(),hits_100['abs_td'].tolist(), hits_1000['abs_td'].tolist() ],
         'Time difference in seconds', './cdf_td.pdf', islogx=True, leg = ['top 10 authors', 'top 100 authors', 'top 1000 authors'])

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
def plot_cdf(list_counts, xlabel, path, leg=False, islogx=True, xlimit=False):
    t_col = "#235dba"
    g_col = "#005916"
    c_col = "#a50808"
    r_col = "#ff9900"
    black = "#000000"
    pink = "#f442f1"
    t_ls = '-'
    r_ls = '--'
    c_ls = ':'
    g_ls = '-.'

    markers = [".", "o", "v", "^", "<", ">", "1", "2"]
    colors = [t_col, c_col, g_col, r_col, black, 'c', 'm', pink]
    line_styles = [t_ls, r_ls, c_ls, g_ls,t_ls, r_ls, c_ls, g_ls, t_ls]
    colors = colors[1:]
    line_styles= line_styles[1:]
    while(len(list_counts) > len(colors)):
        colors = colors + shuffle(colors)
        line_styles = line_styles + shuffle(line_styles)
        
    if xlimit:
        l2 = []
        for l in list_counts:
            l2_1 = [x for x in l if x<=xlimit]
            l2.append(l2_1)
        list_counts = l2
    
    for l in list_counts:
        l.sort()
    fig, ax = plt.subplots(figsize=(6,4))
    yvals = []
    for l in list_counts:
        yvals.append(np.arange(len(l))/float(len(l)-1))
    for i in range(len(list_counts)):
        ax.plot(list_counts[i], yvals[i], color=colors[i], linestyle=line_styles[i])
    if islogx:
        ax.set_xscale("log")
    plt.xlabel(xlabel)
    plt.ylabel('CDF')
    plt.grid()
    for item in ([ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(13)
    
    if leg:
        plt.legend(leg, loc='best', fontsize=13)
    
    plt.show()
    fig.savefig(path, bbox_inches='tight')


In [ ]:
subreddits = hits.groupby(['subreddit'])["post_id"].count().reset_index(name="count")
subreddits = subreddits.sort_values('count', ascending=False)
subreddits.to_csv('./results/top_subreddits_for_top10_authors_rankeed_td_hits.csv')

In [ ]:
complete_hits_2w = complete_hits[(complete_hits['td'] >-1209600) & (complete_hits['td'] < 1209600)]

In [ ]:
subreddits_2w = complete_hits_2w.groupby(['subreddit'])["post_id"].count().reset_index(name="count")
subreddits_2w = subreddits_2w.sort_values('count', ascending=False)

## Baseline of 1000 samples within 5y td for percentage of suspension

In [ ]:
one = pd.read_csv('results/five_year/2016-01.csv')
two = pd.read_csv('./results/five_year/2016-02.csv')

In [ ]:
one=one.append(two, ignore_index = True)

In [ ]:
title_tweetids_df = pd.read_csv('./results/tweet_ids_of_news_titles_tweets.csv')
title_tweetids = title_tweetids_df['tweetid'].tolist()
print('complete hits before removing news titles: ', one.shape)
one_without_news = one[~one['tweet_id'].isin(title_tweetids)]
print('complete hits before after news titles: ', one_without_news.shape)
# Remove authors with name as bot or auto
def is_bot_auto(row):
    author = str(row['post_author'])
    if 'bot' in author:
        return True
    elif 'auto' in author:
        return True
    else:
        return False
one_without_news['is_bot'] = one_without_news.apply(is_bot_auto, axis=1)
one = one_without_news[~one_without_news['is_bot']==True]
print('complete hits after removing bot authors: ', one.shape)

hits_per_author_one = one.groupby(['post_author'])["post_id"].count().reset_index(name="count")
hits_per_author_one = hits_per_author_one.sort_values(by='count', ascending=False)
hits_per_author_one = hits_per_author_one[hits_per_author_one['post_author'] != '[deleted]']


In [ ]:
samples = one.sample(n=10000)
samples = samples[~samples['post_author'].isin(author_status_1w_more_hits['post_author'].tolist())]
# random_samples_author_status = author_status_1y[author_status_1y['post_author']
#                                            .isin(random_samples['post_author'].tolist())]
# print(random_samples_author_status.shape)
# random_samples_1000 = random_samples_author_status.sample(n=1000)

In [ ]:
samples = samples.sample(n=1000)

In [ ]:
import requests
author_status = []
s = 0
for author in samples['post_author'].tolist():
        print(s)
        s = s + 1
        url = 'https://www.reddit.com/user/'+ author +'.json'
        try:
            resp = requests.get(url,headers = {'User-agent': 'your bot 0.1'})
            author_status.append(resp.status_code)
        except Exception as e:
            print(e)
            print(author)
            author_status.append(None)
samples['author_status'] = author_status

In [ ]:
print(samples[samples['author_status'] == 403].shape)
print(samples[samples['author_status'] == 404].shape)
print(samples[samples['author_status'] == 200].shape)
print(samples[samples['author_status'] == 503])
print(samples[samples['author_status'] == 504])
samples['author_status'].unique()

In [ ]:
complete_hits[complete_hits['post_author'] == '_Steve64comments']

## Formatting reddit users

In [ ]:
with open('banned_reddit_users.txt', 'r') as file:
    data = file.read()

In [ ]:
data = data.split('\n')
newdata = []
for l in data:
    l = l.split('|')
    u = l[1].split('/')
    if len(u)>1:
        newdata.append(u[1])
        print(u[1])


In [ ]:
banned_users = pd.DataFrame()
banned_users['author'] = newdata
banned_users.to_csv('./results/banned_suspecious_users.csv')

In [ ]:
# check if users are present in hits that we obtained
complete_hits = pd.DataFrame()
for m in months:
    relevant_hits = pd.read_csv('./results/relevent_hits/'+m+'.csv')
    complete_hits = complete_hits.append(relevant_hits, ignore_index = True)
    print('Number of hits in month ', m, 'with time difference less than a month :', (relevant_hits.shape)[0])

In [ ]:
relevant_hits[relevant_hits['post_author'].isin(newdata)]

In [ ]:
authors = pd.DataFrame()
for m in months:
    relevant_posts = pd.read_csv('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/posts/posts-'+m+'.csv')
    matching_authors = relevant_posts[relevant_posts['author'].isin(newdata)]
    print(m, matching_authors.shape)
    authors = authors.append(matching_authors, ignore_index=True)
#     print(authors)

In [ ]:
authors_hits = pd.DataFrame()

In [ ]:
import json
def process_authors(row):
    auth = row['post_author']
#     if isinstance(auth,str):
#         auth = ast.literal_eval(auth)
    auth = auth.split()
#     print(auth[1])
    return auth[1]
iteration=0
for f in glob.glob('../../reddit_dataset/comments/scores/RC_2017-05.bz2.decompressed/*.txt'):
    iteration = iteration + 1
    with open(f , 'r') as content_file:
        content = content_file.read()
        json_data = content.replace('][',',')
        j_object = json.loads(json_data)
        json_df = pd.DataFrame(j_object)
#         json_df['author'] = json_df.apply(process_authors,axis=1)
        authors_hits = authors_hits.append(json_df[json_df['post_id'].isin(authors['id'].tolist())])
        print(authors_hits.shape)
        print(iteration)



In [ ]:
p_ids = authors_hits['post_id'].unique()

In [ ]:
related_posts = pd.DataFrame()
for m in months:
    relevant_posts = pd.read_csv('/INET/state-trolls/work/state-trolls/reddit_dataset/comments/posts/posts-'+m+'.csv')
    posts = relevant_posts[relevant_posts['id'].isin(p_ids)]

    related_posts = related_posts.append(posts, ignore_index=True)
    print(posts.shape)

In [ ]:
authors_hits.columns

In [ ]:
tweets = pd.read_csv('./relevant_tweets.csv')

In [ ]:
def get_author(row):
    post = related_posts[related_posts['id'] == row['post_id']]
    row['post_author'] = post['author'].tolist()[0]
    row['post_body'] = post['body'].tolist()[0]
    return row
    print(row)
k = authors_hits.apply(get_author,axis=1)
    

In [ ]:
for a in authors['author'].unique():
    s = k[k['post_author']==a]
    print(s['cosine_similarity'].mean())

In [ ]:
t= []
def get_tweet(row):
    t = tweets[tweets['tweetid']==row['tweet_id']]
    return t['tweet_text'].tolist()[0]

In [ ]:
k['tweet_text'] = k.apply(get_tweet, axis=1)

In [ ]:
k.to_csv('./results/hits_suspecious_users.csv')

# get relavent hits by relaxing the threshold but of selected themes

In [ ]:
twitter_data = pd.read_csv('./relevant_tweets.csv')

In [ ]:
title_tweetids_df = pd.read_csv('./results/tweet_ids_of_news_titles_tweets.csv')
title_tweetids = title_tweetids_df['tweetid'].tolist()
print('complete hits before removing news titles: ', complete_hits.shape)
complete_hits_without_news = complete_hits[~complete_hits['tweet_id'].isin(title_tweetids)]
print('complete hits before after news titles: ', complete_hits_without_news.shape)
# Remove authors with name as bot or auto
def is_bot_auto(row):
    author = str(row['post_author'])
    if 'bot' in author:
        return True
    elif 'auto' in author:
        return True
    else:
        return False
complete_hits_without_news['is_bot'] = complete_hits_without_news.apply(is_bot_auto, axis=1)
complete_hits = complete_hits_without_news[~complete_hits_without_news['is_bot']==True]
print('complete hits after removing bot authors: ', complete_hits.shape)

# author status baseline

In [ ]:
aas = author_status_1w_more_hits['post_author'].tolist()

In [ ]:
chunks = pd.DataFrame()
for chunk in pd.read_json('../../reddit_dataset/comments/RC_2016-01.bz2.decompressed',lines = True, chunksize=10000):
    chunks = chunk
    break

In [ ]:
chunks['author'].head()

In [ ]:
random_hits = pd.DataFrame()
for f in glob.glob('./results/baseline_hits/*.csv'):
    df = pd.read_csv(f)
    df = df.drop_duplicates(subset=['post_author'], keep='first')
    random_hits = random_hits.append(df, ignore_index=True)
    print(df.shape)

In [ ]:
print(chunks.shape)
random_hits = chunks.drop_duplicates(subset=['author'], keep='first')
random_hits = chunks[~chunks['author'].isin(aas)]
print(chunks.shape)


In [ ]:
random_set = chunks.sample(1400)

In [ ]:
# def get_author(row):
#     a = row['post_author']
#     auth = a.split()
#     print(auth[1])
#     return auth[1]
# random_set['author'] = random_set.apply(get_author, axis=1)
baseline_authors = random_set['author'].tolist()

In [ ]:
baseline_authors

In [ ]:
statuses = []
count = 0
for author in baseline_authors:
    url = 'https://www.reddit.com/user/'+ author +'.json'
    try:
        resp = requests.get(url,headers = {'User-agent': 'your bot 0.1'})
        statuses.append(resp.status_code)
        count = count + 1
        print('count', count)
    except Exception as e:
        print(e)
        print(author)
        statuses.append(None)

In [ ]:
random_set['author_status'] = statuses

In [ ]:
print(random_set[random_set['author_status']==403].shape)
print(random_set[random_set['author_status']==404].shape)
print(random_set[random_set['author_status']==200].shape)


In [ ]:
random_set['author_status'].unique()